In [ ]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import MeCab
import neologdn
import re
import emoji

In [ ]:
tagger = MeCab.Tagger('') #NEologd辞書指定

tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
#         if lower and node.feature.split(',')[0] in ["名詞","形容詞","副詞"]:#model5
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

    
#学習データの読み込み
df_machie = pd.read_csv('') # レビュー格納csvを指定
df_wash = df_machie.groupby(['prdname'])['comment'].apply(list).apply(' '.join).reset_index().sort_values('prdname')

def remove_unnecessary(text):
    # 半角と全角の統一と重ね表現の除去
    normalized_text = neologdn.normalize(text)
    #URLの除去
    text_without_url = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', normalized_text)
    #桁区切りの除去
    tmp = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text_without_url)
    # 半角記号の置換
    tmp = re.sub(r'[!-/:-@[-`{-~]', r' ', tmp)
    # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
    text_removed_symbol = re.sub(u'[■-♯]', ' ', tmp)
    
    return(text_removed_symbol)


#コーパス作成
wakati_washing_text = []
for i in df_wash['comment']:
    text = remove_unnecessary(i)
    wakati = tokenize(text, 2, 10000, True)
    wakati_washing_text.append(wakati)

wakati_washing_list = np.asarray(wakati_washing_text, dtype = object) 
np.savetxt(" ", wakati_washing_list, fmt = '%s', delimiter = ',', encoding = 'utf-8') #コーパス保存ディレクトリ指定
# np.savetxt("washing_corpus5.txt", wakati_washing_list, fmt = '%s', delimiter = ',', encoding = 'utf-8')

In [ ]:
# モデル作成
word2vec_washing_model = word2vec.Word2Vec(wakati_washing_text, sg = 1, vector_size = 100, window = 5, min_count = 3, epochs = 100, workers = 3)
# モデルのセーブ
word2vec_washing_model.save(" ") #word2vecモデル保存ディレクトリ指定